In [1]:
#!python -m caffe2.python.operator_test.flip_op_test

In [2]:
import numpy as np
from caffe2.python import core, model_helper, workspace, utils
from game import InitGame, AddGamePlay
from modeling import AddConvModel
mini_batch=3

## Init first step

In [3]:
ZERO = np.zeros((mini_batch,1,19,19), dtype=np.float32)
ONE = np.ones((mini_batch,1,19,19), dtype=np.float32)
init_data = np.concatenate((ZERO,ZERO,ONE,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ZERO,ONE), axis=1)
workspace.FeedBlob("data", init_data)

model = model_helper.ModelHelper(name="model", arg_scope={"order": "NCHW"}, init_params=True)
AddConvModel(model, "data", dim_in=13)
AddGamePlay(model, "data", "predict", mini_batch=mini_batch)

workspace.RunNetOnce(model.param_init_net)
workspace.CreateNet(model.net, overwrite=True)
workspace.RunNet(model.net)

init_move = np.reshape(workspace.FetchBlob('predict')[0], (-1)) # shape=(361,)
top_choice = np.argsort(-init_move)[0:mini_batch] # the top K step

for i in range(mini_batch):
    x = top_choice[i]/19
    y = top_choice[i]%19
    init_data[i,1,x,y] = 1 # opponent plus (x,y)
    init_data[i,2,x,y] = 0 # empty minus (x,y)
    init_data[i,4,x,y] = 1 # last 1 step plus (x,y)
    init_data[i,12] = -1

workspace.FeedBlob("data", init_data)

True

In [4]:
for i in range(20):
    workspace.RunNet(model.net)

In [5]:
data=workspace.FetchBlob("data")

In [6]:
data[0,11]

array([[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  4.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.